## CalculateEmbeddings

This script:
- Loads list of sample points, each with 4 associated image files
- Calculates an image embedding for each image file
- Saves a pickle file containig each sample point, the list of image files and the list of embeddings

In [1]:
import os
os.environ["HF_HOME"] = "/nfs/a319/gy17m2a/scratch/hf_cache"
import pickle
import geopandas as gpd
import torch
from PIL import Image
from tqdm import tqdm
import clip

In [ ]:
from directory_filepaths import *

### Load list of sample points
This contains points sampled along the road network in 1-SampleStreetNetwork.iypnb  
Each point has a latitude, a longitude, and 4 image files associated with it (these are sampled in each of the 4 cardinal directions from the sample point)  
It also contains an 'embedding' slot, which this script will fill with a list of embeddings for each of the 4 images

In [2]:
points_data_cache = data_dir + "sample_points_cache/points_data_cache.pkl"
with open(points_data_cache, "rb") as f:
        point_records = pickle.load(f)
print(f"Cache currently has {len(point_records)} points.")

Cache currently has 18897 points.


# Compute the Embeddings

In [3]:
# Define model
device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

## Create embedding for each image and find similarity to categories 
- Create embedding for image
- Find similarity score to text embedding for each category
- Convert similarity score to a "probability-like number" using softmax

In [5]:
def embed_clip(image_path):
    """
    Loads an image, computes its CLIP embedding, 

    Returns:
        image_embedding (np.array): 512-dim CLIP image embedding
    """

    # -----------------------------------------------------------
    # 1. LOAD AND PREPROCESS THE IMAGE
    # -----------------------------------------------------------
    # Load image using PIL and convert to 3-channel RGB
    pil_image = Image.open(image_path).convert("RGB")

    # Apply CLIP preprocessing:
    # - resize/crop to 224x224
    # - convert to torch tensor
    # - normalise pixels with CLIP’s mean/std
    # This produces a tensor of shape (3, 224, 224)
    image_tensor = preprocess(pil_image)

    # Add a batch dimension → (1, 3, 224, 224)
    # Required because CLIP expects a batch
    image_tensor = image_tensor.unsqueeze(0)

    # Move tensor to CPU or GPU depending on device
    image_tensor = image_tensor.to(device)

    # -----------------------------------------------------------
    # 2. RUN CLIP TO GET IMAGE EMBEDDING
    # -----------------------------------------------------------
    # Disable gradient tracking 
    with torch.no_grad():

        # Encode the image → produces a 512-dim CLIP embedding
        raw_image_embedding = model.encode_image(image_tensor)

        # Normalise embedding to unit length (important for cosine similarity)
        image_embedding = raw_image_embedding / raw_image_embedding.norm(
            dim=-1, keepdim=True)
    # -----------------------------------------------------------
    # 4. RETURN CLEAN CPU NUMPY ARRAYS
    # -----------------------------------------------------------
    return image_embedding.cpu().numpy()[0]

# ------------------------------
# 5. Embed all images
# ------------------------------
for rec in tqdm(point_records, desc="Embedding points", unit="point"):
# for rec in point_records[200:300]:    

    rec["embedding"] = []
    rec["category_scores"] = []
    rec["category_probs"] = []
    
    for img_path in rec["image_files"]:

        try:
            embedding = embed_clip(img_path)
            
            rec["embedding"].append(embedding)
            
            # Testing
#             img = Image.open(img_path)
#             fig,ax=plt.subplots(figsize=(2,2))
#             ax.axis("off")
#             plt.imshow(img)
#             plt.show()
#             argmax = scores.argmax()
#             print(list(multi_prompts.keys())[argmax])
            
        except Exception as e:
            tqdm.write(f"⚠️ Error: {e}")

Embedding points:   0%|                                                                                                           | 1/18897 [00:07<39:19:15,  7.49s/point]

KeyboardInterrupt



## Save outputs to pickle file

This contains points sampled along the road network in 1-SampleStreetNetwork.iypnb.    
Each point has a latitude, a longitude, and 4 image files associated with it.  
It also contains an 'embedding' slot, with a list of embeddings for each of the 4 images.  

In [ ]:
output_file = (data_dir + "embeddings/points_data_cache_with_CLIP_embeddings.pkl")

with open(output_file, "wb") as f:
    pickle.dump(point_records, f)

print(f"\n💾 Saved embeddings + category scores for {len(point_records)} points.")

## Testing how CLIP works

In [ ]:
# img_path = point_records[2]["image_files"][3]
# img_path = img_path.replace("airbnb-manchester/", "embeddings/").replace("../", "../../../")
# img = Image.open(img_path)
# plt.imshow(img)

In [ ]:
# from sentence_transformers import SentenceTransformer, util
# from PIL import Image

# #Load CLIP model
# model = SentenceTransformer('clip-ViT-B-32')

In [ ]:
#Encode text descriptions
# text_emb = model.encode(['Two dogs in the snow', "a pizza", 'A cat on a table', 'A picture of a road, with cars and trees'])
# text_emb = model.encode(["a cucumber", "semi-detached house", "a highway with few cars and grass embankments", "a car", "a view down a road", "a park"])

In [ ]:
# #Encode an image:
# img_emb = model.encode(Image.open(img_path))

# #Compute cosine similarities 
# cos_scores = util.cos_sim(img_emb, text_emb)
# print(cos_scores)

In [ ]:
# text_emb = model.encode(["a house", "a shop", "a car", "a road", "a park"])

# #Compute cosine similarities 
# cos_scores = util.cos_sim(img_emb, text_emb)
# print(cos_scores)